# Visualization of Marconi100 jobs power profile

Here i'm using the filtered jobs from the `mem_free` metric. It can have jobs with no power profile but we preserve the exclusive node allocation for the jobs.
I start looking at the multi-node jobs because they are more interesting 

In [1]:
import pandas as pd

df_jobs_viz_multi = pd.read_csv("../example_data/plugin=job_table/metric=job_info_marconi100/metric=mem_free_filter123_multinode.csv")

df_metric = pd.read_parquet("../example_data/plugin=ipmi_pub/metric=total_power/a_0.parquet")
df_metric['node'] = pd.to_numeric(df_metric['node'])
df_metric['value'] = pd.to_numeric(df_metric['value'])

print(df_metric.head(10))

            timestamp  value  node
0 2022-05-14 21:59:00    820   224
1 2022-05-14 21:59:20    540   224
2 2022-05-14 21:59:40    540   224
3 2022-05-13 22:00:00   1280   225
4 2022-05-13 22:00:20   1120   225
5 2022-05-13 22:00:40    940   225
6 2022-05-13 22:01:00    940   225
7 2022-05-13 22:01:20    700   225
8 2022-05-13 22:01:40    800   225
9 2022-05-13 22:02:00   1300   225


In [14]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import matplotlib.pyplot as plt
import ast

#gfig = None



def call_workflow(index,user_id):
    get_job_energy_profile(index, user_id)

def get_job_energy_profile(index, user_id):
    global current_job_id, current_hostname, user_nb_jobs
    annot_str=''
    df_jobs_viz_user = df_jobs_viz_multi.loc[df_jobs_viz_multi["user_id"] == user_id]
    user_nb_jobs = len(df_jobs_viz_user)
    job=df_jobs_viz_user.iloc[index,:]
    nodes=ast.literal_eval(job['nodes'])
    start_time=job["start_time"]
    end_time=job["end_time"]

    current_job_id=str(job['job_id'])
    #for node in nodes:
        #print(node)
    df_node = df_metric.loc[df_metric['node'].isin(nodes)]
    df_node_job = df_node.loc[df_node['timestamp'].between(start_time, end_time)]
    if len(df_node_job) > 0:
        #print(df_node_job.describe(), end="\r")
        plot_energy_profile(df_node_job, job, annot_str)
    #print(nodes)
     
    #plot_energy_profile(job_energy_profile, annot_str)    
    #current_hostname=energy_host
    ##
    #describe=job_energy_profile.describe(percentiles=[.10, .25, .5, .75, .90])
    #describe['job_id']=job['job_id']
    #describe['socket']=socket
    #describe['pp0']=describe[right_pp0]
    #describe['DRAM']=describe[right_DRAM]
    #describe['stat']=describe.index
    #describe=describe.reset_index(drop=True)[arr_cols]  
    #print(describe)  


def plot_energy_profile(profile, job, annot_str):
    TINY_SIZE = 2
    SMALL_SIZE = 5
    MEDIUM_SIZE = 25
    LESS_MEDIUM_SIZE = 30
    LARGE_SIZE = 45
    BIGGER_SIZE = 50
    FIG_WIDTH = 40
    FIG_HEIGHT = 10

    global gfig, gjob, gprofile  

    gjob=job
    gprofile=profile

    plt.rc('font', size=LARGE_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=LARGE_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=LARGE_SIZE)     # fontsize of the x and y labels
    plt.rc('xtick', labelsize=LARGE_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=LARGE_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=LESS_MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    scatterplot_kwargs={'s': 50, 'palette': 'plasma'}
    lineplot_kwargs={'linewidth': 1}

    plt.clf()
    fig = plt.figure(figsize=(FIG_WIDTH,FIG_HEIGHT))
    #ax = sns.boxplot(x='stat', y='value', data=plot_data, showfliers=False, hue='reading',
    #             linewidth=TINY_SIZE)

    #ax = sns.scatterplot(data=profile, x='timestamp', y='value', **scatterplot_kwargs)

    palette = sns.color_palette("viridis", n_colors=job['num_nodes'])
    ax = sns.lineplot(data=profile, x='timestamp', y='value', hue='node', palette=palette, **lineplot_kwargs)
    #ax = sns.scatterplot(data=profile, x='timestamp', y='value', hue='node', palette=palette, **lineplot_kwargs)

    ## SET BORDERS SIZE AND WIDTH
    [line.set_linewidth(TINY_SIZE) for line in ax.spines.values()]
    [line.set_markersize(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.yaxis.get_ticklines()]
    [line.set_markersize(SMALL_SIZE) for line in ax.xaxis.get_ticklines()]
    [line.set_markeredgewidth(TINY_SIZE) for line in ax.xaxis.get_ticklines()]
    #ax.text(x=0.1,y=0.5,
    #        s=annot_str,
    #        fontdict=dict(color='red',size=MEDIUM_SIZE),
    #        bbox=dict(facecolor='yellow',alpha=0.5),
    #        horizontalalignment='left',
    #        verticalalignment='center',
    #        transform=ax.transAxes)
    ax.set_ylabel('Node total power (W)')
    ax.set_xlabel('Timestamp')
    ax.legend(title="Node", loc="upper right")
    #ax.set_title('Job ID: '+current_job_id)    
    gfig = fig

button_saveplot = widgets.Button(description="Save to PDF")
button_savejobinfo = widgets.Button(description="Save job info to CSV")
button_savememinfo = widgets.Button(description="Save mem. profile to CSV")
output = widgets.Output()

display(button_saveplot, button_savejobinfo, button_savememinfo, output)
#display(button_saveplot, output)

## TODO: Pass on b the data (job_id, hostname, etc)
def save_plot_to_pdf_click(b):
    with output:
        #fig=plt.gcf()
        #print(gfig)
        fig_filename='../figures/marconi100_interact_plot_total_power_profile_'+current_job_id+'.pdf'
        gfig.savefig(fig_filename, format='pdf', dpi=300, bbox_inches='tight')
        print('Plot saved as '+fig_filename, end="\r")

def save_job_info_click(b):
    with output:
        job_filename='../samples_jobs_mem_profiles/'+current_job_id+'_job_info.csv'
        pd.DataFrame(gjob).transpose().to_csv(job_filename, index=False, sep=",")        
        print('Job info saved as '+job_filename, end="\r")

def save_mem_profile_click(b):
    with output:
        profile_filename='../samples_jobs_mem_profiles/'+current_job_id+'_mem_profile.csv'
        gprofile.to_csv(profile_filename, index=False, sep=",")
        print('Job mem profile saved as '+profile_filename, end="\r")

button_saveplot.on_click(save_plot_to_pdf_click)
button_savejobinfo.on_click(save_job_info_click)
button_savememinfo.on_click(save_mem_profile_click)

user_ids = df_jobs_viz_multi["user_id"].drop_duplicates().values.tolist()

drp_userids = widgets.Dropdown(
    options=user_ids,
    value=user_ids[0],
    description='User ID: ',
    disabled=False,
)

print(len(df_jobs_viz_multi))
user_nb_jobs = len(df_jobs_viz_multi.loc[df_jobs_viz_multi["user_id"]==user_ids[0]])

slider_index_jobs = widgets.IntSlider(min=0, max=user_nb_jobs-1, step=1, value=0)

#drp_userids.observe(get_job_energy_profile)

# Define a function that updates the intslider's range based on the dropdown selection
def update_range(*args):
    # Update the range of the intslider based on the selected option in the dropdown
    user_nb_jobs = len(df_jobs_viz_multi.loc[df_jobs_viz_multi["user_id"]==drp_userids.value])    
    slider_index_jobs.min = 0
    slider_index_jobs.max = user_nb_jobs-1
    # Reset the intslider's value to the start of the new range
    slider_index_jobs.value = slider_index_jobs.min

drp_userids.observe(update_range, 'value')


interact(call_workflow, index=slider_index_jobs, user_id = drp_userids);

Button(description='Save to PDF', style=ButtonStyle())

Button(description='Save job info to CSV', style=ButtonStyle())

Button(description='Save mem. profile to CSV', style=ButtonStyle())

Output()

966


interactive(children=(IntSlider(value=0, description='index', max=9), Dropdown(description='User ID: ', option…

1. user 212
2. user 262, job 2